In [1]:
import requests
import bs4
import re
import nltk
import fnmatch

root = '/Users/danie/'
base_url = 'https://www.gutenberg.org'

#%%
gutenberg_response_html = requests.get(base_url + '/browse/scores/top')
gutenberg_parsed = bs4.BeautifulSoup(gutenberg_response_html.content, 'html.parser')
location = gutenberg_parsed.find(id='books-last30')
book_listing_tag = location.next_sibling.next_sibling

#%%
def get_bookdata(book_listing, k):
    data = []
    for a_tag in book_listing.find_all('a')[:k]:
        book_name = re.match(r'(.*)(\(\d+\))', a_tag.text)
        book_name = book_name.group(1).strip()
    
        book_id = re.match(r'/ebooks/(\d+)', a_tag.get('href'))
        book_id = book_id.group(1)
    
        book_url = base_url + '/' + book_id + '/'
        
        data.append({"id":book_id, "name":book_name, "url":book_url, "fname":''})
    
    return data

def get_bookdata_byname(book_listing, name):
    data = []
    for a_tag in book_listing.find_all('a'):
        book_name = re.match(r'(.*)(\(\d+\))', a_tag.text)
        book_name = book_name.group(1).strip()
        
        if name not in book_name: continue
        
        book_id = re.match(r'/ebooks/(\d+)', a_tag.get('href'))
        book_id = book_id.group(1)
        
        book_url = base_url + '/' + book_id + '/'
        
        data.append({"id":book_id, "name":book_name, "url":book_url, "fname":''})
        break
    
    return data
        
def complete_urls(bookdata):
    for i in range(len(bookdata)):
        print("Searching txt-file for book with id {}".format(bookdata[i]["id"]))
        indexurl = bookdata[i]["url"]
        
        bookindex_html = requests.get(indexurl)
        bookindex_parsed =  bs4.BeautifulSoup(bookindex_html.content,'html.parser')
        bookindex_links = bookindex_parsed.find_all('a')
        bookindex_hrefs = [bil['href'] for bil in bookindex_links]
        
        book_filenames = [ bih for bih in bookindex_hrefs if fnmatch.fnmatch(bih, '*.txt') ] #.*.txt
        
        bookdata[i]["url"] += book_filenames[0]
        bookdata[i]["fname"] = book_filenames[0]
        
    return

def download_books(bookdata):
    for data in bookdata:
        print("Downloading book: {}".format(data["name"]))
        print("From source: {}".format(data["url"]))
        response = requests.get(data["url"], allow_redirects=True)
        #txtfiles already made
        with open(root + 'txtfiles/' + data["fname"], 'wb') as f:
            f.write(response.content)
            
#%%
book_name = 'The Wonderful Wizard of Oz'
bookdata = get_bookdata_byname(book_listing_tag, book_name)
complete_urls(bookdata)
download_books(bookdata)

#%%
# Download data from local filesystem

def load_data_local(bookdata, fileloc):
    my_text = []
    for data in bookdata:
        try:
            with open(fileloc + data["fname"], 'r') as f:
                my_text.append(f.read())
        except:
            with open(fileloc + data["fname"], 'r', encoding='ISO-8859-1') as f:
                my_text.append(f.read()) 
        print("Book {} loaded.".format(data['id']))
    
    return my_text
    

book_texts = load_data_local(bookdata, root + 'txtfiles/')

#%%
# Remove the start and end information, so only story text is left

def remove_headers(book_texts):
    start_header = '*** START'
    end_header = '*** END'
    new_texts = []
    for text in book_texts:
        start_loc = text.find(start_header)
        print(start_loc)
        start_loc = text[start_loc:].find('\n') + start_loc
        print(start_loc)
        end_loc = text.find(end_header)
        text = text[start_loc : end_loc]
        new_texts.append(text)
        
    return new_texts
        
book_texts = remove_headers(book_texts)
my_text = book_texts[0]

#%% Get the paragraphs
# b)
mytext_paragraphs = paragraphs=re.split('\n[ \n]*\n', my_text)
#paragraphs = paragraphs[1:-1]

Searching txt-file for book with id 55
From source: https://www.gutenberg.org/55/55-0.txt
Book 55 loaded.
717
788


In [2]:
len(mytext_paragraphs)

1141

In [3]:
paragraph_nltk_texts = []
for k in range(len(mytext_paragraphs)):
    temp_tokenizedtext = nltk.word_tokenize(mytext_paragraphs[k])
    temp_nltktext = nltk.Text(temp_tokenizedtext)
    paragraph_nltk_texts.append(temp_nltktext)

In [4]:
paragraph_nltk_texts[0]

<Text: ...>

In [5]:
paragraph_lowercase_texts = []
for k in range(len(paragraph_nltk_texts)):
    temp_lowercase_text = []
    for l in range(len(paragraph_nltk_texts[k])):
        lowercase_word = paragraph_nltk_texts[k][l].lower()
        temp_lowercase_text.append(lowercase_word)
    temp_lowercasetest = nltk.Text(temp_lowercase_text)
    paragraph_lowercase_texts.append(temp_lowercase_text)

In [6]:
#POS
def tagtowordnet(postag):
    wordnettag=-1
    if postag[0]=='N':
        wordnettag='n'
    elif postag[0]=='V':
        wordnettag='v'
    elif postag[0]=='J':
        wordnettag='a'
    elif postag[0]=='R':
        wordnettag='r'
    return(wordnettag)

In [7]:
# Download wordnet resource if you do not have it already
nltk.download('wordnet')
# Download tagger resource if you do not have it already
nltk.download('averaged_perceptron_tagger')

lemmatizer=nltk.stem.WordNetLemmatizer()

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\danie\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\danie\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [8]:
def lemmatizetext(nltktexttolemmatize):
    # Tag the text with POS tags
    taggedtext=nltk.pos_tag(nltktexttolemmatize)
    # Lemmatize each word text
    lemmatizedtext=[]
    for l in range(len(taggedtext)):
        # Lemmatize a word using the WordNet converted POS tag
        wordtolemmatize=taggedtext[l][0]
        wordnettag=tagtowordnet(taggedtext[l][1])
        if wordnettag!=-1:
            lemmatizedword=lemmatizer.lemmatize(wordtolemmatize,wordnettag)
        else:
            lemmatizedword=wordtolemmatize
        # Store the lemmatized word
        lemmatizedtext.append(lemmatizedword)
    return(lemmatizedtext)


# lemmatization of text
#lemmatized_texts = lemmatizetext(lowercase_texts)
#lemmatized_texts = nltk.Text(lemmatized_texts)

In [9]:
paragraph_lemmatizedtexts = []
for k in range(len(paragraph_lowercase_texts)):
    lemmatizedtext = lemmatizetext(paragraph_lowercase_texts[k])
    lemmatizedtext = nltk.Text(lemmatizedtext)
    paragraph_lemmatizedtexts.append(lemmatizedtext)

In [10]:
paragraph_lemmatizedtexts[10]

<Text: have this think in mind , the story...>

In [11]:
import numpy as np
myvocabularies=[]
myindices_in_vocabularies=[]
# Find the vocabulary of each document
for k in range(len(paragraph_lemmatizedtexts)):
    # Get unique words and where they occur
    temptext=paragraph_lemmatizedtexts[k]
    uniqueresults=np.unique(temptext,return_inverse=True)
    uniquewords=uniqueresults[0]
    wordindices=uniqueresults[1]
    # Store the vocabulary and indices of document words in it
    myvocabularies.append(uniquewords)
    myindices_in_vocabularies.append(wordindices)
myvocabularies[0]

array([], dtype=float64)

In [12]:
tempvocabulary=[]
for k in range(len(paragraph_lemmatizedtexts)):
    tempvocabulary.extend(myvocabularies[k])
    
# Find the unique elements among all vocabularies
uniqueresults=np.unique(tempvocabulary,return_inverse=True)
unifiedvocabulary=uniqueresults[0]
wordindices=uniqueresults[1]


In [13]:
# Translate previous indices to unified vocabulary.

vocabularystart=0
myindices_in_unifiedvocabulary=[]
for k in range(len(paragraph_lemmatizedtexts)):
    # In order to shift word indices, we must temporarily
    # change their data type to a Numpy array
    tempindices=np.array(myindices_in_vocabularies[k])
    tempindices=tempindices+vocabularystart
    tempindices=wordindices[tempindices]
    myindices_in_unifiedvocabulary.append(tempindices)
    vocabularystart=vocabularystart+len(myvocabularies[k])

In [14]:
unifiedvocabulary_totaloccurrencecounts=np.zeros((len(unifiedvocabulary),1))
unifiedvocabulary_documentcounts=np.zeros((len(unifiedvocabulary),1))
unifiedvocabulary_meancounts=np.zeros((len(unifiedvocabulary),1))
unifiedvocabulary_countvariances=np.zeros((len(unifiedvocabulary),1))

In [15]:
for k in range(len(paragraph_lemmatizedtexts)):
    print(k)
    occurrencecounts=np.zeros((len(unifiedvocabulary),1))
    for l in range(len(myindices_in_unifiedvocabulary[k])):
        occurrencecounts[myindices_in_unifiedvocabulary[k][l]]= \
            occurrencecounts[myindices_in_unifiedvocabulary[k][l]]+1
    unifiedvocabulary_totaloccurrencecounts= \
        unifiedvocabulary_totaloccurrencecounts+occurrencecounts
    unifiedvocabulary_documentcounts= \
        unifiedvocabulary_documentcounts+(occurrencecounts>0)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [16]:
# Mean occurrence counts over documents
unifiedvocabulary_meancounts= \
    unifiedvocabulary_totaloccurrencecounts/len(paragraph_lemmatizedtexts)

In [17]:
#%% Inspect frequent words
# Sort words by largest total (or mean) occurrence count
highest_totaloccurrences_indices=np.argsort(\
    -1*unifiedvocabulary_totaloccurrencecounts,axis=0)    
print(np.squeeze(unifiedvocabulary[\
    highest_totaloccurrences_indices[0:100]]))
print(np.squeeze(\
    unifiedvocabulary_totaloccurrencecounts[\
    highest_totaloccurrences_indices[0:100]]))

['the' ',' 'and' '.' 'be' 'to' 'of' 'a' 'â\x80\x9d' 'in' 'have' 'i' 'he'
 'you' 'her' 'she' 'they' 'it' 'that' 'say' 'dorothy' 'as' 'so' 'for'
 'with' 'not' 'at' 'but' 'all' 'them' 'do' 'scarecrow' 'his' ';' '?' 'me'
 'him' 'my' 'woodman' 'lion' 'come' 'on' 'will' 'oz' 'great' 'when' 'go'
 'make' 'â\x80\x9ci' 'tin' 'ask' 'little' 'witch' 'this' 'from' 'one'
 'could' 'then' 'see' 'there' 'would' 'we' 'if' 'get' 'up' 'out' 'who'
 'head' 'can' 'green' 'their' 'back' 'look' '!' 'no' 'think' 'girl' 'down'
 'know' 'by' 'toto' 'over' 'answer' 'upon' 'shall' 'find' 'give' 'again'
 'good' 'into' 'very' 'now' 'must' 'city' 'wicked' 'where' 'walk' 'after'
 'emerald' 'long']
[2982. 2703. 1600. 1597. 1433. 1110.  840.  801.  698.  481.  476.  452.
  439.  439.  405.  392.  390.  383.  361.  356.  347.  329.  296.  291.
  274.  272.  251.  243.  239.  233.  231.  219.  216.  196.  194.  186.
  179.  178.  175.  175.  169.  157.  156.  151.  150.  148.  148.  144.
  143.  139.  139.  139.  137.  129.

In [18]:
nltk.download('stopwords')

#%% Vocabulary pruning
nltkstopwords=nltk.corpus.stopwords.words('english')
pruningdecisions=np.zeros((len(unifiedvocabulary),1))
for k in range(len(unifiedvocabulary)):
    # Rule 1: check the nltk stop word list
    if (unifiedvocabulary[k] in nltkstopwords):
        pruningdecisions[k]=1
    # Rule 2: if the word is in the top 1% of frequent words
    #if (k in highest_totaloccurrences_indices[\
    #    0:int(np.floor(len(unifiedvocabulary)*0.01))]):
    #    pruningdecisions[k]=1
   # Rule 3: if the word occurs less than 4 times
    if(unifiedvocabulary_totaloccurrencecounts[k] < 4):
        pruningdecisions[k] = 1
    # Rule 4: if the word is too short
    if len(unifiedvocabulary[k])<2:
        pruningdecisions[k]=1
    # Rule 5: if the word is too long
    if len(unifiedvocabulary[k])>20:
        pruningdecisions[k]=1
    # Rule 6: if the word has unwanted characters
    # (here for simplicity only a-z allowed)
    if unifiedvocabulary[k].isalpha()==False:
        pruningdecisions[k]=1

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\danie\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [19]:
#%% Get indices of documents to remaining words
oldtopruned=[]
tempind=-1
for k in range(len(unifiedvocabulary)):
    if pruningdecisions[k]==0:
        tempind=tempind+1
        oldtopruned.append(tempind)
    else:
        oldtopruned.append(-1)

In [20]:
#%% Create pruned texts
paragraph_prunedtexts=[]
myindices_in_prunedvocabulary=[]
for k in range(len(paragraph_lemmatizedtexts)):
    print(k)
    temp_newindices=[]
    temp_newdoc=[]
    for l in range(len(paragraph_lemmatizedtexts[k])):
        temp_oldindex=myindices_in_unifiedvocabulary[k][l]
        temp_newindex=oldtopruned[temp_oldindex]
        if temp_newindex!=-1:
            temp_newindices.append(temp_newindex)
            temp_newdoc.append(unifiedvocabulary[temp_oldindex])
    paragraph_prunedtexts.append(temp_newdoc)
    myindices_in_prunedvocabulary.append(temp_newindices)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [21]:
print('Top 100 word-list after pruning unified vocabulary:\n')
remaining_indices = np.squeeze(np.where(pruningdecisions==0)[0])
remaining_vocabulary = unifiedvocabulary[remaining_indices]
remainingvocabulary_totaloccurrencecounts = unifiedvocabulary_totaloccurrencecounts[remaining_indices]
remaining_highest_totaloccurrences_indices = np.argsort(-1*remainingvocabulary_totaloccurrencecounts, axis=0)
print(np.squeeze(remaining_vocabulary[remaining_highest_totaloccurrences_indices[0:100]]))
print(np.squeeze(remainingvocabulary_totaloccurrencecounts[remaining_highest_totaloccurrences_indices[0:100]]))

Top 100 word-list after pruning unified vocabulary:

['say' 'dorothy' 'scarecrow' 'woodman' 'lion' 'come' 'oz' 'great' 'go'
 'make' 'tin' 'ask' 'little' 'witch' 'one' 'could' 'see' 'would' 'get'
 'head' 'green' 'back' 'look' 'think' 'girl' 'know' 'toto' 'answer' 'upon'
 'find' 'shall' 'give' 'good' 'must' 'city' 'wicked' 'walk' 'emerald'
 'long' 'man' 'country' 'room' 'tree' 'away' 'heart' 'like' 'take' 'big'
 'time' 'way' 'carry' 'tell' 'saw' 'people' 'us' 'never' 'eye' 'reply'
 'stand' 'monkey' 'brain' 'live' 'well' 'many' 'chapter' 'day' 'forest'
 'first' 'run' 'road' 'ever' 'friend' 'soon' 'help' 'house' 'around'
 'much' 'keep' 'wish' 'wizard' 'arm' 'cry' 'beast' 'sit' 'thing' 'old'
 'mouse' 'call' 'land' 'beautiful' 'shoe' 'leave' 'air' 'woman' 'seem'
 'put' 'fly' 'quite' 'voice' 'begin']
[356. 347. 219. 175. 175. 169. 151. 150. 148. 144. 139. 139. 139. 137.
 122. 120. 119. 113. 105. 101. 100.  99.  97.  94.  94.  93.  90.  86.
  85.  81.  81.  81.  80.  75.  75.  72.  71.  69.  6

In [22]:
import scipy
#%% Create TF-IDF vectors
n_docs=len(paragraph_prunedtexts)
n_vocab=len(remaining_vocabulary)
# Matrix of term frequencies
tfmatrix=scipy.sparse.lil_matrix((n_docs,n_vocab))
# Row vector of document frequencies
dfvector=scipy.sparse.lil_matrix((1,n_vocab))
# Loop over documents
for k in range(n_docs):
    # Row vector of which words occurred in this document
    temp_dfvector=scipy.sparse.lil_matrix((1,n_vocab))
    # Loop over words
    for l in range(len(paragraph_prunedtexts[k])):
        # Add current word to term-frequency count and document-count
        currentword=myindices_in_prunedvocabulary[k][l]
        tfmatrix[k,currentword]=tfmatrix[k,currentword]+1
        temp_dfvector[0,currentword]=1
    # Add which words occurred in this document to overall document counts
    dfvector=dfvector+temp_dfvector

# TF:length-normalized frequency
for i in range(n_docs):
    for j in range(len(tfmatrix.data[i])):
        tfmatrix.data[i][j] = tfmatrix.data[i][j]/len(tfmatrix.data[i])
        

# smoothed logarithmic idf
idfvector = np.squeeze(np.array(dfvector.todense()))
idfvector = np.log(1 + ((idfvector+1)**-1)*n_docs)        

tfidfmatrix = scipy.sparse.lil_matrix((n_docs, n_vocab))
for k in range(n_docs):
    # tf and idf terms
    tfidfmatrix[k,:]=tfmatrix[k,:]*idfvector
    
# tf-idf matrix
#tfidfmatrix = scipy.sparse.lil_matrix((n_docs, n_vocab))
for k in range(n_docs):
    # find nonzero term frequencies
    tempindices = np.nonzero(tfmatrix[k, :])[1]
    tfterm = np.squeeze(np.array(tfmatrix[k, tempindices].todense()))
    tfidfmatrix[k, tempindices] = tfterm * idfvector[tempindices]

C:\Users\danie\Anaconda3\lib\site-packages\scipy\sparse\lil.py:512: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not i.flags.writeable or i.dtype not in (np.int32, np.int64):
C:\Users\danie\Anaconda3\lib\site-packages\scipy\sparse\lil.py:514: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not j.flags.writeable or j.dtype not in (np.int32, np.int64):
C:\Users\danie\Anaconda3\lib\site-packages\scipy\sparse\lil.py:518: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not x.flags.writeable:


In [23]:
#%% Use the TF-IDF matrix as data to be clustered
X=tfidfmatrix
# Normalize the documents to unit vector norm
tempnorms=np.squeeze(np.array(np.sum(X.multiply(X),axis=1)))
# If any documents have zero norm, avoid dividing them by zero
tempnorms[tempnorms==0]=1
X=scipy.sparse.diags(tempnorms**-0.5).dot(X)
n_data=np.shape(X)[0]
n_dimensions=np.shape(X)[1]

In [24]:
# Function to initialize the Gaussian mixture model, create component parameters
def initialize_mixturemodel(X,n_components):
    # Create lists of sparse matrices to hold the parameters
    n_dimensions=np.shape(X)[1]
    n_data = np.shape(X)[0]
    mixturemodel_means=scipy.sparse.lil_matrix((n_components,n_dimensions))
    mixturemodel_weights=np.zeros((n_components))
    mixturemodel_covariances=[]
    mixturemodel_inversecovariances=[]
    for k in range(n_components):
        tempcovariance=scipy.sparse.lil_matrix((n_dimensions,n_dimensions))
        mixturemodel_covariances.append(tempcovariance)
        tempinvcovariance=scipy.sparse.lil_matrix((n_dimensions,n_dimensions))
        mixturemodel_inversecovariances.append(tempinvcovariance)
    # Initialize the parameters
    for k in range(n_components):
        mixturemodel_weights[k]=1/n_components
        # Pick a random data point as the initial mean
        tempindex=scipy.stats.randint.rvs(low=0,high=n_data)
        mixturemodel_means[k]=X[tempindex,:].toarray()
        # Initialize the covariance matrix to be spherical
        for l in range(n_dimensions):
            mixturemodel_covariances[k][l,l]=1
            mixturemodel_inversecovariances[k][l,l]=1
    return(mixturemodel_weights,mixturemodel_means,mixturemodel_covariances,mixturemodel_inversecovariances)

In [25]:
def run_estep(X,mixturemodel_means,mixturemodel_covariances,mixturemodel_inversecovariances,mixturemodel_weights):
    # For each component, compute terms that do not involve data
    meanterms=np.zeros((n_components))
    logdeterminants=np.zeros((n_components))
    logconstantterms=np.zeros((n_components))
    
    for k in range(n_components):
        # Compute mu_k*inv(Sigma_k)*mu_k
        meanterms[k]=(mixturemodel_means[k,:]*mixturemodel_inversecovariances[k]*mixturemodel_means[k,:].T)[0,0]
        # Compute determinant of Sigma_k. For a diagonal matrix
        # this is just the product of the main diagonal
        logdeterminants[k]=np.sum(np.log(mixturemodel_covariances[k].diagonal(0)))
        # Compute constant term beta_k * 1/(|Sigma_k|^1/2)
        # Omit the (2pi)^d/2 as it cancels out
        logconstantterms[k]=np.log(mixturemodel_weights[k]) - 0.5*logdeterminants[k]
    
    print('E-step part2 ')
    # Compute terms that involve distances of data from components
    xnorms=np.zeros((n_data,n_components))
    xtimesmu=np.zeros((n_data,n_components))
    
    for k in range(n_components):
        #print(k)
        xnorms[:,k]=(X*mixturemodel_inversecovariances[k]*X.T).diagonal(0)
        xtimesmu[:,k]=np.squeeze((X*mixturemodel_inversecovariances[k]* mixturemodel_means[k,:].T).toarray())
        
    xdists=xnorms+np.matlib.repmat(meanterms,n_data,1)-2*xtimesmu
    # Substract maximal term before exponent (cancels out) to maintain computational precision
    numeratorterms=logconstantterms-xdists/2
    numeratorterms-=np.matlib.repmat(np.max(numeratorterms,axis=1),n_components,1).T
    numeratorterms=np.exp(numeratorterms)
    mixturemodel_componentmemberships=numeratorterms/np.matlib.repmat(np.sum(numeratorterms,axis=1),n_components,1).T
    return(mixturemodel_componentmemberships)

In [26]:
def run_mstep_sumweights(mixturemodel_componentmemberships):
    # Compute total weight per component
    mixturemodel_weights=np.sum(mixturemodel_componentmemberships,axis=0)
    return(mixturemodel_weights)

In [27]:
def run_mstep_means(X,mixturemodel_componentmemberships,mixturemodel_weights):
    # Update component means
    mixturemodel_means=scipy.sparse.lil_matrix((n_components,n_dimensions))
    for k in range(n_components):
        mixturemodel_means[k,:]=\
            np.sum(scipy.sparse.diags(mixturemodel_componentmemberships[:,k]).dot(X),axis=0)
        mixturemodel_means[k,:]/=mixturemodel_weights[k]
    return(mixturemodel_means)

In [28]:
def run_mstep_covariances(X,mixturemodel_componentmemberships,mixturemodel_weights,mixturemodel_means):
    # Update diagonal component covariance matrices
    n_dimensions=np.shape(X)[1]
    n_components=np.shape(mixturemodel_componentmemberships)[1]
    tempcovariances=np.zeros((n_components,n_dimensions))
    mixturemodel_covariances=[]
    mixturemodel_inversecovariances=[]
    
    for k in range(n_components):
        tempcovariances[k,:]= np.sum(scipy.sparse.diags(
            mixturemodel_componentmemberships[:,k]).dot(
            X.multiply(X)),axis=0)-mixturemodel_means[k,:].multiply(mixturemodel_means[k,:])*mixturemodel_weights[k]
        tempcovariances[k,:]/=mixturemodel_weights[k]
        # Convert to sparse matrices
        tempepsilon=1e-10
        # Add a small regularization term
        temp_covariance=scipy.sparse.diags(tempcovariances[k,:]+tempepsilon)
        temp_inversecovariance=scipy.sparse.diags((tempcovariances[k,:]+tempepsilon)**-1)
        mixturemodel_covariances.append(temp_covariance)
        mixturemodel_inversecovariances.append(temp_inversecovariance)
    return(mixturemodel_covariances,mixturemodel_inversecovariances)

In [29]:
def run_mstep_normalizeweights(mixturemodel_weights):
    # Update mixture-component prior probabilities
    mixturemodel_weights/=sum(mixturemodel_weights)
    return(mixturemodel_weights)

In [30]:
#%% Perform the EM algorithm iterations
def perform_emalgorithm(X,n_components,n_emiterations):
    mixturemodel_weights,mixturemodel_means,mixturemodel_covariances,\
    mixturemodel_inversecovariances=initialize_mixturemodel(X,n_components)
    for t in range(n_emiterations):
        # ====== E-step: Compute the component membership
        # probabilities of each data point ======
        print('E-step ' + str(t))
        mixturemodel_componentmemberships=run_estep(X,mixturemodel_means,mixturemodel_covariances,\
        mixturemodel_inversecovariances,mixturemodel_weights)
        # ====== M-step: update component parameters======
        print('M-step ' + str(t))
        print('M-step part1 ' + str(t))
        mixturemodel_weights=run_mstep_sumweights(mixturemodel_componentmemberships)
        print('M-step part2 ' + str(t))
        mixturemodel_means=run_mstep_means(X,mixturemodel_componentmemberships,mixturemodel_weights)
        print('M-step part3 ' + str(t))
        mixturemodel_covariances,mixturemodel_inversecovariances=run_mstep_covariances(X,\
        mixturemodel_componentmemberships,mixturemodel_weights,mixturemodel_means)
        print('M-step part4 ' + str(t))
        mixturemodel_weights=run_mstep_normalizeweights(mixturemodel_weights)
    return(mixturemodel_weights,mixturemodel_means,mixturemodel_covariances,\
mixturemodel_inversecovariances)
# Try out the functions we just defined on the data
n_components=10
n_emiterations=100
mixturemodel_weights,mixturemodel_means,mixturemodel_covariances,\
mixturemodel_inversecovariances = perform_emalgorithm(X,n_components,n_emiterations)

C:\Users\danie\Anaconda3\lib\site-packages\scipy\sparse\lil.py:512: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not i.flags.writeable or i.dtype not in (np.int32, np.int64):
C:\Users\danie\Anaconda3\lib\site-packages\scipy\sparse\lil.py:514: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not j.flags.writeable or j.dtype not in (np.int32, np.int64):


E-step 0
E-step part2 
M-step 0
M-step part1 0
M-step part2 0
M-step part3 0
M-step part4 0
E-step 1
E-step part2 
M-step 1
M-step part1 1
M-step part2 1
M-step part3 1
M-step part4 1
E-step 2
E-step part2 
M-step 2
M-step part1 2
M-step part2 2
M-step part3 2
M-step part4 2
E-step 3
E-step part2 
M-step 3
M-step part1 3
M-step part2 3
M-step part3 3
M-step part4 3
E-step 4
E-step part2 
M-step 4
M-step part1 4
M-step part2 4
M-step part3 4
M-step part4 4
E-step 5
E-step part2 
M-step 5
M-step part1 5
M-step part2 5
M-step part3 5
M-step part4 5
E-step 6
E-step part2 
M-step 6
M-step part1 6
M-step part2 6
M-step part3 6
M-step part4 6
E-step 7
E-step part2 
M-step 7
M-step part1 7
M-step part2 7
M-step part3 7
M-step part4 7
E-step 8
E-step part2 
M-step 8
M-step part1 8
M-step part2 8
M-step part3 8
M-step part4 8
E-step 9
E-step part2 
M-step 9
M-step part1 9
M-step part2 9
M-step part3 9
M-step part4 9
E-step 10
E-step part2 
M-step 10
M-step part1 10
M-step part2 10
M-step part3 1

M-step 84
M-step part1 84
M-step part2 84
M-step part3 84
M-step part4 84
E-step 85
E-step part2 
M-step 85
M-step part1 85
M-step part2 85
M-step part3 85
M-step part4 85
E-step 86
E-step part2 
M-step 86
M-step part1 86
M-step part2 86
M-step part3 86
M-step part4 86
E-step 87
E-step part2 
M-step 87
M-step part1 87
M-step part2 87
M-step part3 87
M-step part4 87
E-step 88
E-step part2 
M-step 88
M-step part1 88
M-step part2 88
M-step part3 88
M-step part4 88
E-step 89
E-step part2 
M-step 89
M-step part1 89
M-step part2 89
M-step part3 89
M-step part4 89
E-step 90
E-step part2 
M-step 90
M-step part1 90
M-step part2 90
M-step part3 90
M-step part4 90
E-step 91
E-step part2 
M-step 91
M-step part1 91
M-step part2 91
M-step part3 91
M-step part4 91
E-step 92
E-step part2 
M-step 92
M-step part1 92
M-step part2 92
M-step part3 92
M-step part4 92
E-step 93
E-step part2 
M-step 93
M-step part1 93
M-step part2 93
M-step part3 93
M-step part4 93
E-step 94
E-step part2 
M-step 94
M-step par

In [31]:
for k in range(n_components):
    print(k)
    highest_dimensionweight_indices=np.argsort(-np.squeeze(mixturemodel_means[k,:].toarray()),axis=0)

    print(' '.join(remaining_vocabulary[highest_dimensionweight_indices[1:10]]))

0
look supper lonely suppose sure surely listen surprise surprised
1
lucky magic tinsmith man manage mane low many mark
2
pat pave perch person pick piece pin pass place
3
pole fairy prairie presently pull everywhere purple everyone push
4
inside inquire sweep immediately imagine swiftly table hour hot
5
others ought palace part party pat patch order pave
6
hung humbug however story hour hot horse hope home
7
oblige offer often oil old open order ought paint
8
marry search ruby careful ought name joyfully chapter sadly
9
fairy nearly whirl explain expect exclaim need move exactly


In [32]:
# Version 2 - Get documents closest to component mean, i.e. highest p(d|k).
# ---The computation of distances here is the same as done in the E-step of EM---
# For each component, compute terms that do not involve data
meanterms=np.zeros((n_components))
logdeterminants=np.zeros((n_components))
logconstantterms=np.zeros((n_components))
for k in range(n_components):
    # Compute mu_k*inv(Sigma_k)*mu_k
    meanterms[k]=(mixturemodel_means[k,:]*mixturemodel_inversecovariances[k]*mixturemodel_means[k,:].T)[0,0]

# Compute terms that involve distances of data from components
xnorms=np.zeros((n_data,n_components))
xtimesmu=np.zeros((n_data,n_components))

for k in range(n_components):
    xnorms[:,k]=(X*mixturemodel_inversecovariances[k]*X.T).diagonal(0)
    xtimesmu[:,k]=np.squeeze((X*mixturemodel_inversecovariances[k]*mixturemodel_means[k,:].T).toarray())

xdists=xnorms+np.matlib.repmat(meanterms,n_data,1)-2*xtimesmu

for k in range(n_components):
    tempdists=np.array(np.squeeze(xdists[:,k]))
    highest_componentprob_indices=np.argsort(-1*tempdists,axis=0)
    print(k)
    print(highest_componentprob_indices[0:10])
    print(' '.join(paragraph_nltk_texts[highest_componentprob_indices[0]]))

0
[   0  876  635  633  631  420   11 1140    1    7]

1
[   0  876  635  633  631  420   11 1140    7    3]

2
[   0  876  635  633  631  420   11 1140    3    1]

3
[   0  876  635  633  631  420   11 1140    7    3]

4
[   0  876  635  633  631  420   11 1140    7    1]

5
[   0  876  635  633  631  420   11 1140    7    3]

6
[   0  876  635  633  631  420   11 1140    3    7]

7
[   0  876  635  633  631  420   11 1140    7    1]

8
[   0  876  635  633  631  420   11 1140    1    7]

9
[   0  876  635  633  631  420   11 1140    3    7]



# EX 4.2

In [33]:
#Find out the amout of words in paragraphs
word_count = np.zeros((len(paragraph_lemmatizedtexts), 1))
for k in range(len(paragraph_lemmatizedtexts)):
    counting = len(paragraph_lemmatizedtexts[k])
    word_count[k] = counting
    

In [34]:
max(word_count)

array([234.])

In [35]:
no_longest_paragraph = np.argsort(-1*word_count, axis=0)[0]


In [36]:
no_longest_paragraph

array([505], dtype=int64)

In [37]:
#The longest paragraph 
print(' '.join(paragraph_lemmatizedtexts[505]))

she leave dorothy alone and go back to the others . these she also lead to room , and each one of them find himself lodge in a very pleasant part of the palace . of course this politeness be waste on the scarecrow ; for when he find himself alone in his room he stand stupidly in one spot , just within the doorway , to wait till morning . it would not rest him to lie down , and he could not close his eye ; so he remain all night star at a little spider which be weave its web in a corner of the room , just as if it be not one of the most wonderful room in the world . the tin woodman lay down on his bed from force of habit , for he remember when he be make of flesh ; but not be able to sleep , he pass the night move his joint up and down to make sure they keep in good work order . the lion would have prefer a bed of dried leaf in the forest , and do not like be shut up in a room ; but he have too much sense to let this worry him , so he spring upon the bed and roll himself up like a cat a

In [41]:
longestpara=(' '.join(paragraph_lemmatizedtexts[505]))
longestpara


'she leave dorothy alone and go back to the others . these she also lead to room , and each one of them find himself lodge in a very pleasant part of the palace . of course this politeness be waste on the scarecrow ; for when he find himself alone in his room he stand stupidly in one spot , just within the doorway , to wait till morning . it would not rest him to lie down , and he could not close his eye ; so he remain all night star at a little spider which be weave its web in a corner of the room , just as if it be not one of the most wonderful room in the world . the tin woodman lay down on his bed from force of habit , for he remember when he be make of flesh ; but not be able to sleep , he pass the night move his joint up and down to make sure they keep in good work order . the lion would have prefer a bed of dried leaf in the forest , and do not like be shut up in a room ; but he have too much sense to let this worry him , so he spring upon the bed and roll himself up like a cat 

In [42]:
longestmyvocabularies=[]
longestmyindices_in_vocabularies=[]
# Find the vocabulary of each document
for k in range(len(longestpara)):
    # Get unique words and where they occur
    temptext=longestpara[k]
    uniqueresults=np.unique(temptext,return_inverse=True)
    uniquewords=uniqueresults[0]
    wordindices=uniqueresults[1]
    # Store the vocabulary and indices of document words in it
    longestmyvocabularies.append(uniquewords)
    longestmyindices_in_vocabularies.append(wordindices)
longestmyvocabularies[0]

array(['s'], dtype='<U1')

In [43]:
tempvocabulary=[]
for k in range(len(paragraph_lemmatizedtexts)):
    tempvocabulary.extend(myvocabularies[k])
    
# Find the unique elements among all vocabularies
uniqueresults=np.unique(tempvocabulary,return_inverse=True)
unifiedvocabulary=uniqueresults[0]
wordindices=uniqueresults[1]

In [44]:
# Translate previous indices to unified vocabulary.

vocabularystart=0
myindices_in_unifiedvocabulary=[]
for k in range(len(paragraph_lemmatizedtexts)):
    # In order to shift word indices, we must temporarily
    # change their data type to a Numpy array
    tempindices=np.array(myindices_in_vocabularies[k])
    tempindices=tempindices+vocabularystart
    tempindices=wordindices[tempindices]
    myindices_in_unifiedvocabulary.append(tempindices)
    vocabularystart=vocabularystart+len(myvocabularies[k])

In [45]:
unifiedvocabulary_totaloccurrencecounts=np.zeros((len(unifiedvocabulary),1))
unifiedvocabulary_documentcounts=np.zeros((len(unifiedvocabulary),1))
unifiedvocabulary_meancounts=np.zeros((len(unifiedvocabulary),1))
unifiedvocabulary_countvariances=np.zeros((len(unifiedvocabulary),1))

In [46]:
for k in range(len(longestpara)):
    print(k)
    occurrencecounts=np.zeros((len(unifiedvocabulary),1))
    for l in range(len(myindices_in_unifiedvocabulary[k])):
        occurrencecounts[myindices_in_unifiedvocabulary[k][l]]= \
            occurrencecounts[myindices_in_unifiedvocabulary[k][l]]+1
    unifiedvocabulary_totaloccurrencecounts= \
        unifiedvocabulary_totaloccurrencecounts+occurrencecounts
    unifiedvocabulary_documentcounts= \
        unifiedvocabulary_documentcounts+(occurrencecounts>0)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [47]:
# Mean occurrence counts over documents
unifiedvocabulary_meancounts= \
    unifiedvocabulary_totaloccurrencecounts/len(longestpara)

In [48]:
#%% Inspect frequent words
# Sort words by largest total (or mean) occurrence count
highest_totaloccurrences_indices=np.argsort(\
    -1*unifiedvocabulary_totaloccurrencecounts,axis=0)    
print(np.squeeze(unifiedvocabulary[\
    highest_totaloccurrences_indices[0:100]]))
print(np.squeeze(\
    unifiedvocabulary_totaloccurrencecounts[\
    highest_totaloccurrences_indices[0:100]]))

['the' ',' '.' 'and' 'be' 'to' 'of' 'a' 'â\x80\x9d' 'in' 'have' 'i' 'he'
 'you' 'her' 'she' 'they' 'it' 'that' 'say' 'dorothy' 'as' 'so' 'for'
 'not' 'with' 'at' 'but' 'do' 'all' 'them' 'scarecrow' 'his' ';' '?' 'him'
 'my' 'me' 'woodman' 'come' 'on' 'lion' 'oz' 'great' 'make' 'will' 'go'
 'â\x80\x9ci' 'when' 'little' 'tin' 'witch' 'ask' 'this' 'one' 'could'
 'from' 'see' 'would' 'then' 'there' 'we' 'if' 'who' 'get' 'green' 'out'
 'up' 'can' 'their' 'head' 'look' 'know' 'no' 'think' 'girl' 'back' 'toto'
 '!' 'down' 'by' 'upon' 'answer' 'find' 'shall' 'again' 'city' 'give'
 'into' 'over' 'very' 'must' 'wicked' 'now' 'emerald' 'man' 'good' 'where'
 'after' 'walk']
[2709. 2494. 1485. 1458. 1324. 1009.  760.  737.  633.  441.  424.  422.
  410.  401.  369.  361.  357.  356.  342.  323.  321.  290.  282.  278.
  256.  252.  237.  228.  217.  217.  212.  203.  198.  185.  178.  172.
  172.  170.  168.  153.  151.  150.  148.  139.  139.  138.  137.  136.
  136.  133.  133.  128.  126.  118. 

In [49]:
nltk.download('stopwords')

#%% Vocabulary pruning
nltkstopwords=nltk.corpus.stopwords.words('english')
pruningdecisions=np.zeros((len(unifiedvocabulary),1))
for k in range(len(unifiedvocabulary)):
    # Rule 1: check the nltk stop word list
    if (unifiedvocabulary[k] in nltkstopwords):
        pruningdecisions[k]=1
    # Rule 2: if the word is in the top 1% of frequent words
    #if (k in highest_totaloccurrences_indices[\
    #    0:int(np.floor(len(unifiedvocabulary)*0.01))]):
    #    pruningdecisions[k]=1
   # Rule 3: if the word occurs less than 4 times
    if(unifiedvocabulary_totaloccurrencecounts[k] < 4):
        pruningdecisions[k] = 1
    # Rule 4: if the word is too short
    if len(unifiedvocabulary[k])<2:
        pruningdecisions[k]=1
    # Rule 5: if the word is too long
    if len(unifiedvocabulary[k])>20:
        pruningdecisions[k]=1
    # Rule 6: if the word has unwanted characters
    # (here for simplicity only a-z allowed)
    if unifiedvocabulary[k].isalpha()==False:
        pruningdecisions[k]=1

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\danie\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [50]:
#%% Get indices of documents to remaining words
longestoldtopruned=[]
tempind=-1
for k in range(len(unifiedvocabulary)):
    if pruningdecisions[k]==0:
        tempind=tempind+1
        longestoldtopruned.append(tempind)
    else:
        longestoldtopruned.append(-1)

In [53]:
#%% Create pruned texts
longestparagraph_prunedtexts=[]
longestmyindices_in_prunedvocabulary=[]
for k in range(len(longestpara)):
    print(k)
    temp_newindices=[]
    temp_newdoc=[]
    for l in range(len(longestpara[k])):
        temp_oldindex=myindices_in_unifiedvocabulary[k][l]
        temp_newindex=longestoldtopruned[temp_oldindex]
        if temp_newindex!=-1:
            temp_newindices.append(temp_newindex)
            temp_newdoc.append(unifiedvocabulary[temp_oldindex])
    longestparagraph_prunedtexts.append(temp_newdoc)
    longestmyindices_in_prunedvocabulary.append(temp_newindices)

0


IndexError: index 0 is out of bounds for axis 0 with size 0

In [54]:
print('Top 100 word-list after pruning unified vocabulary:\n')
remaining_indices = np.squeeze(np.where(pruningdecisions==0)[0])
remaining_vocabulary = unifiedvocabulary[remaining_indices]
remainingvocabulary_totaloccurrencecounts = unifiedvocabulary_totaloccurrencecounts[remaining_indices]
remaining_highest_totaloccurrences_indices = np.argsort(-1*remainingvocabulary_totaloccurrencecounts, axis=0)
print(np.squeeze(remaining_vocabulary[remaining_highest_totaloccurrences_indices[0:100]]))
print(np.squeeze(remainingvocabulary_totaloccurrencecounts[remaining_highest_totaloccurrences_indices[0:100]]

SyntaxError: unexpected EOF while parsing (<ipython-input-54-aed8dc634baa>, line 7)

In [55]:
n_docs=len(longestparagraph_prunedtexts)
n_vocab=len(remaining_vocabulary)
# Matrix of term frequencies
tfmatrix=scipy.sparse.lil_matrix((n_docs,n_vocab))
# Row vector of document frequencies
dfvector=scipy.sparse.lil_matrix((1,n_vocab))
# Loop over documents
for k in range(n_docs):
    # Row vector of which words occurred in this document
    temp_dfvector=scipy.sparse.lil_matrix((1,n_vocab))
    # Loop over words
    for l in range(len(paragraph_prunedtexts[k])):
        # Add current word to term-frequency count and document-count
        currentword=longestmyindices_in_prunedvocabulary[k][l]
        tfmatrix[k,currentword]=tfmatrix[k,currentword]+1
        temp_dfvector[0,currentword]=1
    # Add which words occurred in this document to overall document counts
    dfvector=dfvector+temp_dfvector

# TF:length-normalized frequency
for i in range(n_docs):
    for j in range(len(tfmatrix.data[i])):
        tfmatrix.data[i][j] = tfmatrix.data[i][j]/len(tfmatrix.data[i])
        

# smoothed logarithmic idf
idfvector = np.squeeze(np.array(dfvector.todense()))
idfvector = np.log(1 + ((idfvector+1)**-1)*n_docs)        

tfidfmatrix = scipy.sparse.lil_matrix((n_docs, n_vocab))
for k in range(n_docs):
    # tf and idf terms
    tfidfmatrix[k,:]=tfmatrix[k,:]*idfvector
    
# tf-idf matrix
#tfidfmatrix = scipy.sparse.lil_matrix((n_docs, n_vocab))
for k in range(n_docs):
    # find nonzero term frequencies
    tempindices = np.nonzero(tfmatrix[k, :])[1]
    tfterm = np.squeeze(np.array(tfmatrix[k, tempindices].todense()))
    tfidfmatrix[k, tempindices] = tfterm * idfvector[tempindices]

In [56]:
#%% Use the TF-IDF matrix as data to be clustered
X=tfidfmatrix
# Normalize the documents to unit vector norm
tempnorms=np.squeeze(np.array(np.sum(X.multiply(X),axis=1)))
# If any documents have zero norm, avoid dividing them by zero
tempnorms[tempnorms==0]=1
X=scipy.sparse.diags(tempnorms**-0.5).dot(X)
n_data=np.shape(X)[0]
n_dimensions=np.shape(X)[1]

In [57]:
# Function to initialize the Gaussian mixture model, create component parameters
def initialize_mixturemodel(X,n_components):
    # Create lists of sparse matrices to hold the parameters
    n_dimensions=np.shape(X)[1]
    n_data = np.shape(X)[0]
    mixturemodel_means=scipy.sparse.lil_matrix((n_components,n_dimensions))
    mixturemodel_weights=np.zeros((n_components))
    mixturemodel_covariances=[]
    mixturemodel_inversecovariances=[]
    for k in range(n_components):
        tempcovariance=scipy.sparse.lil_matrix((n_dimensions,n_dimensions))
        mixturemodel_covariances.append(tempcovariance)
        tempinvcovariance=scipy.sparse.lil_matrix((n_dimensions,n_dimensions))
        mixturemodel_inversecovariances.append(tempinvcovariance)
    # Initialize the parameters
    for k in range(n_components):
        mixturemodel_weights[k]=1/n_components
        # Pick a random data point as the initial mean
        tempindex=scipy.stats.randint.rvs(low=0,high=n_data)
        mixturemodel_means[k]=X[tempindex,:].toarray()
        # Initialize the covariance matrix to be spherical
        for l in range(n_dimensions):
            mixturemodel_covariances[k][l,l]=1
            mixturemodel_inversecovariances[k][l,l]=1
    return(mixturemodel_weights,mixturemodel_means,mixturemodel_covariances,mixturemodel_inversecovariances)

In [58]:
def run_estep(X,mixturemodel_means,mixturemodel_covariances,mixturemodel_inversecovariances,mixturemodel_weights):
    # For each component, compute terms that do not involve data
    meanterms=np.zeros((n_components))
    logdeterminants=np.zeros((n_components))
    logconstantterms=np.zeros((n_components))
    
    for k in range(n_components):
        # Compute mu_k*inv(Sigma_k)*mu_k
        meanterms[k]=(mixturemodel_means[k,:]*mixturemodel_inversecovariances[k]*mixturemodel_means[k,:].T)[0,0]
        # Compute determinant of Sigma_k. For a diagonal matrix
        # this is just the product of the main diagonal
        logdeterminants[k]=np.sum(np.log(mixturemodel_covariances[k].diagonal(0)))
        # Compute constant term beta_k * 1/(|Sigma_k|^1/2)
        # Omit the (2pi)^d/2 as it cancels out
        logconstantterms[k]=np.log(mixturemodel_weights[k]) - 0.5*logdeterminants[k]
    
    print('E-step part2 ')
    # Compute terms that involve distances of data from components
    xnorms=np.zeros((n_data,n_components))
    xtimesmu=np.zeros((n_data,n_components))
    
    for k in range(n_components):
        #print(k)
        xnorms[:,k]=(X*mixturemodel_inversecovariances[k]*X.T).diagonal(0)
        xtimesmu[:,k]=np.squeeze((X*mixturemodel_inversecovariances[k]* mixturemodel_means[k,:].T).toarray())
        
    xdists=xnorms+np.matlib.repmat(meanterms,n_data,1)-2*xtimesmu
    # Substract maximal term before exponent (cancels out) to maintain computational precision
    numeratorterms=logconstantterms-xdists/2
    numeratorterms-=np.matlib.repmat(np.max(numeratorterms,axis=1),n_components,1).T
    numeratorterms=np.exp(numeratorterms)
    mixturemodel_componentmemberships=numeratorterms/np.matlib.repmat(np.sum(numeratorterms,axis=1),n_components,1).T
    return(mixturemodel_componentmemberships)

In [59]:
def run_mstep_sumweights(mixturemodel_componentmemberships):
    # Compute total weight per component
    mixturemodel_weights=np.sum(mixturemodel_componentmemberships,axis=0)
    return(mixturemodel_weights)

In [60]:
def run_mstep_means(X,mixturemodel_componentmemberships,mixturemodel_weights):
    # Update component means
    mixturemodel_means=scipy.sparse.lil_matrix((n_components,n_dimensions))
    for k in range(n_components):
        mixturemodel_means[k,:]=\
            np.sum(scipy.sparse.diags(mixturemodel_componentmemberships[:,k]).dot(X),axis=0)
        mixturemodel_means[k,:]/=mixturemodel_weights[k]
    return(mixturemodel_means)

In [61]:
def run_mstep_covariances(X,mixturemodel_componentmemberships,mixturemodel_weights,mixturemodel_means):
    # Update diagonal component covariance matrices
    n_dimensions=np.shape(X)[1]
    n_components=np.shape(mixturemodel_componentmemberships)[1]
    tempcovariances=np.zeros((n_components,n_dimensions))
    mixturemodel_covariances=[]
    mixturemodel_inversecovariances=[]
    
    for k in range(n_components):
        tempcovariances[k,:]= np.sum(scipy.sparse.diags(
            mixturemodel_componentmemberships[:,k]).dot(
            X.multiply(X)),axis=0)-mixturemodel_means[k,:].multiply(mixturemodel_means[k,:])*mixturemodel_weights[k]
        tempcovariances[k,:]/=mixturemodel_weights[k]
        # Convert to sparse matrices
        tempepsilon=1e-10
        # Add a small regularization term
        temp_covariance=scipy.sparse.diags(tempcovariances[k,:]+tempepsilon)
        temp_inversecovariance=scipy.sparse.diags((tempcovariances[k,:]+tempepsilon)**-1)
        mixturemodel_covariances.append(temp_covariance)
        mixturemodel_inversecovariances.append(temp_inversecovariance)
    return(mixturemodel_covariances,mixturemodel_inversecovariances)

In [62]:
def run_mstep_normalizeweights(mixturemodel_weights):
    # Update mixture-component prior probabilities
    mixturemodel_weights/=sum(mixturemodel_weights)
    return(mixturemodel_weights)

In [63]:
#%% Perform the EM algorithm iterations
def perform_emalgorithm(X,n_components,n_emiterations):
    mixturemodel_weights,mixturemodel_means,mixturemodel_covariances,\
    mixturemodel_inversecovariances=initialize_mixturemodel(X,n_components)
    for t in range(n_emiterations):
        # ====== E-step: Compute the component membership
        # probabilities of each data point ======
        print('E-step ' + str(t))
        mixturemodel_componentmemberships=run_estep(X,mixturemodel_means,mixturemodel_covariances,\
        mixturemodel_inversecovariances,mixturemodel_weights)
        # ====== M-step: update component parameters======
        print('M-step ' + str(t))
        print('M-step part1 ' + str(t))
        mixturemodel_weights=run_mstep_sumweights(mixturemodel_componentmemberships)
        print('M-step part2 ' + str(t))
        mixturemodel_means=run_mstep_means(X,mixturemodel_componentmemberships,mixturemodel_weights)
        print('M-step part3 ' + str(t))
        mixturemodel_covariances,mixturemodel_inversecovariances=run_mstep_covariances(X,\
        mixturemodel_componentmemberships,mixturemodel_weights,mixturemodel_means)
        print('M-step part4 ' + str(t))
        mixturemodel_weights=run_mstep_normalizeweights(mixturemodel_weights)
    return(mixturemodel_weights,mixturemodel_means,mixturemodel_covariances,\
mixturemodel_inversecovariances)
# Try out the functions we just defined on the data
n_components=10
n_emiterations=100
mixturemodel_weights,mixturemodel_means,mixturemodel_covariances,\
mixturemodel_inversecovariances = perform_emalgorithm(X,n_components,n_emiterations)

ValueError: Domain error in arguments.

In [64]:
for k in range(n_components):
    print(k)
    highest_dimensionweight_indices=np.argsort(-np.squeeze(mixturemodel_means[k,:].toarray()),axis=0)

    print(' '.join(remaining_vocabulary[highest_dimensionweight_indices[1:10]]))

0
look supper lonely suppose sure surely listen surprise surprised
1
lucky magic tinsmith man manage mane low many mark
2
pat pave perch person pick piece pin pass place
3
pole fairy prairie presently pull everywhere purple everyone push
4
inside inquire sweep immediately imagine swiftly table hour hot
5
others ought palace part party pat patch order pave
6
hung humbug however story hour hot horse hope home
7
oblige offer often oil old open order ought paint
8
marry search ruby careful ought name joyfully chapter sadly
9
fairy nearly whirl explain expect exclaim need move exactly


In [65]:
# Version 2 - Get documents closest to component mean, i.e. highest p(d|k).
# ---The computation of distances here is the same as done in the E-step of EM---
# For each component, compute terms that do not involve data
meanterms=np.zeros((n_components))
logdeterminants=np.zeros((n_components))
logconstantterms=np.zeros((n_components))
for k in range(n_components):
    # Compute mu_k*inv(Sigma_k)*mu_k
    meanterms[k]=(mixturemodel_means[k,:]*mixturemodel_inversecovariances[k]*mixturemodel_means[k,:].T)[0,0]

# Compute terms that involve distances of data from components
xnorms=np.zeros((n_data,n_components))
xtimesmu=np.zeros((n_data,n_components))

for k in range(n_components):
    xnorms[:,k]=(X*mixturemodel_inversecovariances[k]*X.T).diagonal(0)
    xtimesmu[:,k]=np.squeeze((X*mixturemodel_inversecovariances[k]*mixturemodel_means[k,:].T).toarray())

xdists=xnorms+np.matlib.repmat(meanterms,n_data,1)-2*xtimesmu

for k in range(n_components):
    tempdists=np.array(np.squeeze(xdists[:,k]))
    highest_componentprob_indices=np.argsort(-1*tempdists,axis=0)
    print(k)
    print(highest_componentprob_indices[0:10])
    print(' '.join(paragraph_nltk_texts[highest_componentprob_indices[0]]))

ValueError: k exceeds matrix dimensions